# Challenge 1 Notebook

In [3]:
import pandas as pd
import random
import itertools
from IPython.display import display, HTML


## Prisoner's Dilemma Game function

In [2]:
# Function to simulate a prisoner's dilemma game
def play_prisoners_dilemma(contestants, iterations=5):
    # Initialize a DataFrame to store results
    results = pd.DataFrame(columns=['Contestant1', 'Contestant2', 'Round', 'C1_Decision', 'C2_Decision', 'C1_Payoff', 'C2_Payoff'])

    # Define payoff matrix
    payoff_matrix = {
        ('Cooperate', 'Cooperate'): (2, 2),
        ('Cooperate', 'Defect'): (0, 3),
        ('Defect', 'Cooperate'): (3, 0),
        ('Defect', 'Defect'): (1, 1)
    }

    # Initialize a dictionary to store total scores
    total_scores = {contestant: 0 for contestant in contestants.keys()}

    # Initialize a dictionary to store matchup scores
    matchup_scores = {c1: {c2: 0 for c2 in contestants.keys()} for c1 in contestants.keys()}

    # Iterate over all pairs of contestants
    for c1, c2 in itertools.combinations(contestants.keys(), 2):
        # Initialize history DataFrames for both contestants
        c1_history = pd.DataFrame(columns=['Round', 'Opponent_Decision', 'My_Decision'])
        c2_history = pd.DataFrame(columns=['Round', 'Opponent_Decision', 'My_Decision'])

        for round_number in range(1, iterations + 1):
            # Get decisions from both contestants
            c1_decision = contestants[c1](c1_history)
            c2_decision = contestants[c2](c2_history)

            # Record decisions in results DataFrame
            c1_payoff, c2_payoff = payoff_matrix[(c1_decision, c2_decision)]
            new_result = pd.DataFrame([{
                'Contestant1': c1,
                'Contestant2': c2,
                'Round': round_number,
                'C1_Decision': c1_decision,
                'C2_Decision': c2_decision,
                'C1_Payoff': c1_payoff,
                'C2_Payoff': c2_payoff
            }])
            results = pd.concat([results, new_result], ignore_index=True)

            # Update total scores
            total_scores[c1] += c1_payoff
            total_scores[c2] += c2_payoff

            # Update matchup scores
            matchup_scores[c1][c2] += c1_payoff
            matchup_scores[c2][c1] += c2_payoff

            # Update history DataFrames
            new_c1_history = pd.DataFrame([{'Round': round_number, 'Opponent_Decision': c2_decision, 'My_Decision': c1_decision}])
            new_c2_history = pd.DataFrame([{'Round': round_number, 'Opponent_Decision': c1_decision, 'My_Decision': c2_decision}])
            c1_history = pd.concat([c1_history, new_c1_history], ignore_index=True)
            c2_history = pd.concat([c2_history, new_c2_history], ignore_index=True)

    return results, total_scores, matchup_scores



## Contestant Functions

In [4]:
# Contestant strategies
def pushover(history):
    return 'Cooperate'

def gaslighter(history):
    """
    Cooperates the first three turns, then defects thereafter.
    """
    if history.empty or len(history) < 3:
        return 'Cooperate'
    return 'Defect'

def north_korea(history):
    """
    Defects the first five times, then cooperates the next five times, and so on.
    """
    # Determine the current round
    current_round = len(history) + 1

    # Determine the phase of the strategy
    phase = (current_round - 1) // 5

    if phase % 2 == 0:
        return 'Defect'
    else:
        return 'Cooperate'

def Sam(history):
    if not history.empty:
        if history.Round.max() == 1 or history.Round.max() == 4:
            return "Cooperate"
    return "Defect"

ryan = lambda history: 'Defect'

def Steve(history):
    if not history.empty:
        if list(history.sort_values(by="Round").Opponent_Decision)[:5] == ["Defect", "Cooperate", "Defect", "Defect", "Cooperate"]:
            return "Cooperate"
    return "Defect"

def wenng(history):
    """
    truly a horrible mindset but it is what it is in a cut throat world...
    """
    if not history.empty:
        if 'Defect' in history['Opponent_Decision'].values:
            return 'Defect'
        else:
            return 'Cooperate'
    return 'Cooperate'

def fat_finger(history):
    import random
    choice_list = ['Cooperate', 'Defect']
    wts = [1,19]
    choice = random.choices(choice_list, weights=wts)
    return(choice[0])

def Con_Air(history):
    return 'Defect'

def Saad(history):
    return 'Defect'

def punisher(history):
    current_round = history.shape[0] + 1
    if current_round == 1:
        return "Cooperate"
    elif current_round < 50:
        opp_decs = set(history.Opponent_Decision)
        if "Cooperate" in opp_decs and len(opp_decs) == 1:
            return "Cooperate"
        else:
            return "Defect"
    else:
        return "Defect"

def jon_k(history):
    if not history.empty:
        if list(history.sort_values(by="Round").Opponent_Decision)[:5] == ["Defect", "Cooperate", "Defect", "Defect", "Cooperate"]:
            return "Cooperate"
    return "Defect"

def paffshank_redemption(df):
    #Overly complicated to make it interesting

    # First round
    if df.empty:
        return 'Cooperate'

    current_round = df['Round'].iloc[-1] + 1
    rounds_left = 50 - current_round

    opponent_history = df['Opponent_Decision'].tolist()
    my_history = df['My_Decision'].tolist()

    # End game strat
    # Always defect in the last x rounds
    if rounds_left < 1:
        return 'Defect'

    # Always defect if opponent defected in first two rounds - no mercy
    if current_round > 2 and opponent_history[:2] == ['Defect', 'Defect']:
        return 'Defect'

    # Adaptive defection - add some spice
    # Take advantage of forgiving functions
    # Try first defection on round 10(increase?)
    if current_round == 10:
        return 'Defect'

    # If not punished for defection increase frequency of defection
    unpunished_defections = 0
    for i in range(len(my_history) - 1):
        if my_history[i] == 'Defect' and opponent_history[i + 1] == 'Cooperate':
            unpunished_defections += 1
    if unpunished_defections > 0:
        defection_frequency = max(10 - unpunished_defections, 2)  # Frequency ranges from 2 to 10
        if current_round % defection_frequency == 0:
            return 'Defect'

    # Attempt Pattern recognition
    # Opponent might be alternating
    if opponent_history[-5:] == ['Cooperate', 'Defect'] * 2 + ['Cooperate']:
        return 'Defect'

    # Check if we're in a tit-for-tat feedback loop
    def in_alternating_pattern():
        if len(my_history) >= 4:
            if my_history[-4:] == ['Defect', 'Cooperate', 'Defect', 'Cooperate']:
                return True
        return False

    # Punish if opponent has ever defected after we attempted to reconcile
    def opponent_defected_after_cooperation():
        for i in range(len(my_history) - 1):
            if my_history[i] == 'Cooperate' and opponent_history[i + 1] == 'Defect':
                return True
        return False

    # Anti-bounce mechanism
    # Attempt to break the cycle by cooperating
    if current_round > 4 and in_alternating_pattern():
        # If opponent has ever defected after we cooperated, no forgiveness
        if opponent_defected_after_cooperation():
            return 'Defect'
        else:
            return 'Cooperate'

    # Mid-game strat - make it more interesting
    # Punitive against frequent Defectors
    cooperation_ratio = opponent_history.count('Cooperate') / len(opponent_history)

    # Adjust cooperation threshold based on rounds left
    cooperation_threshold = 0.7 + (rounds_left / 50) * 0.1

    if cooperation_ratio < 0.3:
        return 'Defect'

    # Tit-for-tat as a fallback strat zzzzzz
    return opponent_history[-1]

def neo_mar(history):
    if not history.empty:
        if list(history.sort_values(by="Round").Opponent_Decision)[:5] == ["Defect", "Cooperate", "Defect", "Defect", "Cooperate"]:
            return "Cooperate"
    return "Defect"

def Julia(history):
    import numpy as np
    import sys
    np.set_printoptions(threshold=sys.maxsize)
    np.set_printoptions(linewidth=999999)


##    import matplotlib.pyplot as plt

    N_rounds = 5
    if history.empty == 1:
        my_fraction = np.random.random()
        their_fraction = np.random.random()
    else:
        MAX = max(history['Round'])
        MIN = np.clip(MAX - N_rounds, a_min=0, a_max=None)

##        print(history.loc[
##            (history['Round'] >= MIN) & (history['Round'] <= MAX),
##            :
##        ])

        my_fraction = sum(history.loc[
            (history['Round'] >= MIN) & (history['Round'] <= MAX),
            'My_Decision'
        ] == 'Cooperate')/(MAX - MIN)
        my_fraction += np.random.normal(0, 0.1)
        my_fraction = np.clip(my_fraction, a_min=0, a_max=1)

        their_fraction = sum(history.loc[
            (history['Round'] >= MIN) & (history['Round'] <= MAX),
            'Opponent_Decision'
        ] == 'Cooperate')/(MAX - MIN)

    if (my_fraction == 1) | (my_fraction == 0):
        my_fraction = np.random.random()

    c = complex(1.5*(my_fraction-0.5), 1.5*(their_fraction-0.5))

    w = 200
    h = 200
    re_min, re_max = -1.5, 1.5
    im_min, im_max = -1.5, 1.5
    real_range = np.arange(re_min, re_max, (re_max - re_min) / w)
    imag_range = np.arange(im_max, im_min, (im_min - im_max) / h)

    point_x = w*their_fraction + np.random.normal(0, 10)
    point_x = np.clip(int(round(point_x)), a_min=0, a_max=w)
    point_y = h*my_fraction + np.random.normal(0, 10)
    point_y = np.clip(int(round(point_y)), a_min=0, a_max=h)

    img = np.ones([w+1, h+1])*255
    for y, im in enumerate(imag_range):
        for x, re in enumerate(real_range):
            z = complex(re, im)
            n = 255
            while abs(z) < 10 and n >= 5:
                z = z*z + c
                n -= 5
            img[y, x] = n

    if img[point_y, point_x] == 0:
        result = "Cooperate"
    else:
        result = "Defect"

    return result

def defect(history):
    return 'Defect'

def M_Strategy(history):
    if not len(history) < 2:
        last_opp_decision = history.iloc[-1]['Opponent_Decision']
        second_last_opp_decision=history.iloc[-2]['Opponent_Decision']
        my_last=history.iloc[-1]['My_Decision']
        if last_opp_decision == 'Cooperate' and second_last_opp_decision=='Cooperate':
            return 'Defect'
        if last_opp_decision=='Cooperate' and my_last=='Defect':
            return 'Defect'
        elif last_opp_decision == 'Defect' and second_last_opp_decision == 'Defect':
            return 'Cooperate'
        elif last_opp_decision == 'Cooperate':
            return 'Cooperate'
        elif last_opp_decision == 'Defect':
            return 'Defect'
        pass
    return 'Cooperate'

#### not a python coder, so hope this works as intended
#### tit-for-tat strategy with two modifications:
#### A: always defects on round 50
#### B: if faces two consecutive defections after round 40 then behaves 'spitefully' and defects for remainder of match.

def eneguE(history):
    if(history.empty):
        return('Cooperate')
    elif(history['Round'].iloc[-1]==49):
        return('Defect')
    elif(history['Round'].iloc[-1]>39 and history['My_Decision'].iloc[-1] =='Defect' and history['My_Decision'].iloc[-2] =='Defect' ):
        return('Defect')
    else:
        return(history['Opponent_Decision'].iloc[-1])

def grudge_holder(history):
    if history.empty:
        return "Cooperate"
    if (history["Opponent_Decision"] == "Defect").any():
        return "Defect"
    return "Cooperate"

def sus_tit_for_tat(history):
    return 'Defect' if history.empty else history['Opponent_Decision'].iloc[-1]


def my_adaptive_strategy(history):
    # If less than 10 rounds have been played, cooperate
    if len(history) < 10:
        return 'Cooperate'
    else:
        # Convert 'Cooperate' to 1 and 'Defect' to 0 for numeric operations
        numeric_opponent_decisions = history['Opponent_Decision'].map({'Cooperate': 1, 'Defect': 0})

        # Calculate the cooperation rate of the opponent
        cooperation_rate = numeric_opponent_decisions.mean()

        # Check if the opponent is playing tit-for-tat
        if len(history) > 1 and (history['My_Decision'].shift() == history['Opponent_Decision']).mean() > 0.8:
            # If the opponent is likely playing tit-for-tat, always cooperate to maximise points
            return 'Cooperate'
        else:
            # Calculate the average and standard deviation of the numeric opponent decisions
            avg_cooperation_rate = numeric_opponent_decisions.mean()
            std_cooperation_rate = numeric_opponent_decisions.std()

            # If the opponent is ultra cooperative (cooperation rate > average cooperation rate + standard deviation), cooperate
            if cooperation_rate > avg_cooperation_rate + std_cooperation_rate:
                return 'Cooperate'
            # If the opponent is neutral (average cooperation rate - standard deviation < cooperation rate < average cooperation rate + standard deviation), repeat the opponent's last move
            elif avg_cooperation_rate - std_cooperation_rate < cooperation_rate < avg_cooperation_rate + std_cooperation_rate:
                return history.iloc[-1]['Opponent_Decision']
            # If the opponent is ultra aggressive (cooperation rate < average cooperation rate - standard deviation), do whatever will maximise our points
            else:
                # If the opponent's last decision was to cooperate, cooperate to get 2 points
                if history.iloc[-1]['Opponent_Decision'] == 'Cooperate':
                    return 'Cooperate'
                # If the opponent's last decision was to defect, defect to get 1 point
                else:
                    return 'Defect'

def boaty_mcboatface(history):
    '''
    Boaty McBoatface is a 6-state markov chain whose parameters were derived using an evolutionary algorithm from
    the Axelrod dojo. Born an raised in a dark-mode jupyter-notebook, Boaty yearns for all
    the honour, glory and riches typically bestowed upon expert IPD players. Boaty is generally good natured
    and loves to cooperate, however they have been known to exploit opponents who are unwilling to defend
    themselves. Ironically, their forgiving nature and slow response to attack makes them are quite susceptible
    to exploitation themselves.

    Thanks for organising JS :D
    mark.ioppolo@abs.gov.au
    '''

    import random
    states = range(6)

    # MC transition matrices
    tc = [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
         [0.2152918268699183, 0.0, 0.5711235577222001, 0.0, 0.1756423176553314, 0.03794229775255],
         [0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
         [0.7562996085061778, 0.12984048726976818, 0.11385990422405398, 0.0, 0.0, 0.0],
         [0.0, 0.0, 0.621205733868114, 0.36641226577677666, 0.0, 0.01238200035510953],
         [0.0, 0.0, 0.13322873866188784, 0.2903615376776261, 0.4098980322042518, 0.16651169145623426]]

    td = [[0.0, 0.0, 0.026076633941006053, 0.0, 0.9739233660589939, 0.0],
         [0.20527026584837002, 0.5109502437779226, 0.07320930918744979, 0.0, 0.21057018118625748, 0.0],
         [0.0, 0.0, 0.7627695361034562, 0.23723046389654387, 0.0, 0.0],
         [0.4485687201890345, 0.0, 0.0, 0.47501605272958364, 0.07641522708138196, 0.0],
         [0.0, 0.14280703446680046, 0.8206371135534533, 0.0, 0.03655585197974629, 0.0],
         [0.5938511219375519, 0.05828461836000699, 0.17715237368756548, 0.0, 0.17071188601487564, 0.0]]

    # Probability of defecting given state==index
    prob = [1, 0.8881378694567235, 0, 1, 0, 0.1072007322053194]

    # Look at match history and determine the current state
    # Initial state is 2
    state = 2

    if not history.empty:
        for i in range(len(history)):
            # for each row, look at the opponents move, look up transition probs, use probs to determine next state
            last_play = history['Opponent_Decision'][i]

            if last_play == 'Cooperate':
                transition_probs = tc[state]
            elif last_play == 'Defect':
                transition_probs = td[state]
            else:
                print('https://en.wikipedia.org/wiki/RRS_Sir_David_Attenborough#Naming_poll')

            state = random.choices(states, weights=transition_probs, k=1)[0]

    # I've now updated my state. I'll just figure out my response.
    response = random.choices(['Cooperate', 'Defect'], weights = [1-prob[state], prob[state]], k=1)[0]
    return response


# Define contestants
contestants = {
    'gaslighter': gaslighter,
    'pushover': pushover,
    'north korea': north_korea,
    # 'Sam': Sam,
    'ryan': ryan,
    # 'Steve': Steve,
    'wenng': wenng,
    'fat_finger': fat_finger,
    'Con Air': Con_Air,
    'Saad': Saad,
    'punisher': punisher,
    # 'jon k': jon_k,
    'paffshank redemption': paffshank_redemption,
    # 'neo mar': neo_mar,
    'Julia Fractal': Julia,
    'ASR': defect,
    'MLN': M_Strategy,
    'eneguE': eneguE,
    'grudge holder': grudge_holder,
    'coolpant': sus_tit_for_tat,
    'JFK': my_adaptive_strategy,
    'Boaty McBoatface': boaty_mcboatface
}

# Run the prisoner's dilemma simulation
results, total_scores, matchup_scores = play_prisoners_dilemma(contestants, iterations=50)



## Function Categories
### Mystery Functions


1.   pushover: Always cooperates
2.   gaslighter: Cooperates for 3 rounds, then defects until end
3. north korea: Defects for 5 rounds, then cooperates for 5 rounds in a loop

### Nasty functions


1.   ryan: Always defects
2.   Con Air: Always defects
3. Saad: Always defects
4. fat_finger: 95% chance of defecting, 5% chance of cooperating (fat-finger?)
5. ASR: Always defects

### One strike


1.   wenng: Cooperates unless 'defect' in opponent.history

> wenng: truly a horrible mindset but it is what it is in a cut throat world...

2.   punisher: Cooperates unless 'defect' in opponent.history + defects on final round
3. grudge_holder: Cooperates unless 'defect' in opponent.history

### Tit-For-Tat
1. eneguE: Returns the last decision made by the opponent + defects on final round
2. coolpant: Returns the last decision made by the opponent, but starts with 'Defect'
3. JFK: Initial cooperation for first 10 rounds to build a history of the opponents decisions and calculate cooperation rate. Uses an adaptive strategy based on the opponent's behaviour to maximise points.

### Contrarian
1. MLN: When it detects two cooperations, it will defect. But if it detects two defections, it will cooperate! Slightly masochistic =/

### Complex functions
1. paffshank_redemption: 77 lines! Multiple helper functions, pattern recognition, fallback positions, punishments for opponents who defect after reconciliation, anti-bounce mechanisms, cooperation ratio analysis, and more!
     
> Jacob: Overly complicated to make it interesting

2. Julia Fractal: 66 lines! This strategy combines randomness and fractal geometry to decide whether to cooperate or defect. Initially, the decisions are random, but as rounds progress, they are influenced by the recent history of the player's and opponent's cooperation rates. It would make it hard for opponents to predict the player behaviour accurately.
3. Boaty_McBoatface:
> Boaty McBoatface is a 6-state markov chain whose parameters were derived using an evolutionary algorithm from the Axelrod dojo. Born an raised in a dark-mode jupyter-notebook, Boaty yearns for all the honour, glory and riches typically bestowed upon expert IPD players. Boaty is generally good natured    and loves to cooperate, however they have been known to exploit opponents who are unwilling to defend themselves. Ironically, their forgiving nature and slow response to attack makes them are quite susceptible to exploitation themselves.


---



### The Mafia
1. Sam: Don
2. Steve: Soldier
3. jon k: Soldier
4. neo mar: Soldier








## Display Results Matrix (scores)

In [5]:
# Display the score matrix with hyphens for self-intersections
matchup_scores_df = pd.DataFrame(matchup_scores).astype(int).T

# Function to apply color scale
def color_scale(val, row_name, col_name):
    if row_name == col_name:
        return 'background-color: black'
    # Define the range for the colors
    min_val, max_val = 0, matchup_scores_df.max().max()
    range_val = max_val - min_val
    if range_val == 0:
        return 'background-color: rgb(255,255,255)'  # Avoid division by zero
    # Calculate the color intensity based on the value
    intensity = (val - min_val) / range_val
    # Convert the intensity to a color
    r = int(255 * (1 - intensity))
    g = int(255 * intensity)
    b = 0
    return f'background-color: rgb({r},{g},{b})'

# Apply the color scale to the entire DataFrame
def apply_color_scale(df):
    styled_df = df.style.apply(lambda col: [
        color_scale(df.loc[idx, col.name], idx, col.name) for idx in df.index], axis=0)
    return styled_df

# Apply the color scale function to the DataFrame
styled_df = apply_color_scale(matchup_scores_df)

# Display the styled DataFrame
display(HTML("<h2>Matchup Scores</h2>"))
display(styled_df)


,gaslighter,pushover,north korea,ryan,wenng,fat_finger,Con Air,Saad,punisher,paffshank redemption,Julia Fractal,ASR,MLN,eneguE,grudge holder,coolpant,JFK,Boaty McBoatface
gaslighter,0,147,97,47,55,53,47,47,55,55,49,47,141,55,55,53,71,109
pushover,6,0,50,0,100,6,0,0,98,76,0,0,4,98,100,98,100,100
north korea,31,125,0,25,27,35,25,25,27,27,41,25,79,67,27,73,85,101
ryan,56,150,100,0,52,56,50,50,52,52,50,50,150,52,52,50,70,118
wenng,52,100,99,49,0,55,49,49,98,60,56,49,141,98,100,51,100,100
fat_finger,53,147,95,47,49,0,47,50,50,49,50,47,147,55,47,53,75,106
Con Air,56,150,100,50,52,56,0,50,52,52,50,50,150,52,52,50,70,124
Saad,56,150,100,50,52,50,50,0,52,52,52,50,150,52,52,50,70,118
punisher,52,101,99,49,101,53,49,49,0,60,49,49,141,99,101,51,101,101
paffshank redemption,52,112,99,49,60,55,49,49,60,0,73,49,83,83,60,55,92,100


## Display Results Matrix (win/draw/loss)

In [6]:
# Create a new DataFrame for W/D/L results
w_d_l_df = pd.DataFrame(index=matchup_scores_df.index, columns=matchup_scores_df.columns)

# Calculate W/D/L results
for c1 in matchup_scores_df.columns:
    for c2 in matchup_scores_df.index:
        if c1 == c2:
            w_d_l_df.at[c2, c1] = '-'
        else:
            if matchup_scores_df.at[c2, c1] > matchup_scores_df.at[c1, c2]:
                w_d_l_df.at[c2, c1] = 'W'
            elif matchup_scores_df.at[c2, c1] < matchup_scores_df.at[c1, c2]:
                w_d_l_df.at[c2, c1] = 'L'
            else:
                w_d_l_df.at[c2, c1] = 'D'

# Define a function to color-code the W/D/L values
def wdl_color(val):
    color_map = {'W': 'background-color: green', 'D': 'background-color: yellow', 'L': 'background-color: red', '-': 'background-color: black'}
    return color_map.get(val, '')

# Apply the color scale function to the W/D/L DataFrame
styled_w_d_l_df = w_d_l_df.style.applymap(wdl_color)

# Display the styled DataFrame
display(HTML("<h2>Win / Draw / Loss Results</h2>"))
display(styled_w_d_l_df)

,gaslighter,pushover,north korea,ryan,wenng,fat_finger,Con Air,Saad,punisher,paffshank redemption,Julia Fractal,ASR,MLN,eneguE,grudge holder,coolpant,JFK,Boaty McBoatface
gaslighter,-,W,W,L,W,D,L,L,W,W,L,L,W,W,W,D,W,W
pushover,L,-,L,L,D,L,L,L,L,L,L,L,L,L,D,L,D,D
north korea,L,W,-,L,L,L,L,L,L,L,L,L,W,L,L,L,W,W
ryan,W,W,W,-,W,W,D,D,W,W,D,D,W,W,W,D,W,W
wenng,L,D,W,L,-,W,L,L,L,D,W,L,W,L,D,D,D,D
fat_finger,D,W,W,L,L,-,L,D,L,L,D,L,W,W,L,D,W,W
Con Air,W,W,W,D,W,W,-,D,W,W,D,D,W,W,W,D,W,W
Saad,W,W,W,D,W,D,D,-,W,W,W,D,W,W,W,D,W,W
punisher,L,W,W,L,W,W,L,L,-,D,L,L,W,D,W,D,W,W
paffshank redemption,L,W,W,L,D,W,L,L,D,-,W,L,W,D,D,D,W,W


## Total Ranked Scores

In [7]:
# Calculate the total score for each contestant
total_scores = matchup_scores_df.sum(axis=1)
total_scores.name = 'Total_Score'

# Create a new series with the contestant names and their total scores
total_scores_series = pd.Series(total_scores, name='Total_Score').sort_values(ascending=False)

# Display the total scores
display(HTML("<h2>Total Scores</h2>"))
display(total_scores_series.to_frame().style.apply(lambda x: [
    'background-color: gold' if x.name == total_scores_series.idxmax() else
    'background-color: silver' if x.name == total_scores_series.nlargest(2).idxmin() else
    'background-color: #cd7f32' if x.name == total_scores_series.nlargest(3).idxmin() else ''
], axis=1))

,Total_Score
grudge holder,1308
wenng,1306
punisher,1305
eneguE,1277
Con Air,1216
ASR,1210
ryan,1210
Saad,1206
JFK,1196
Julia Fractal,1187


## Mafia Round

In [8]:

# Define contestants
contestants = {
    'gaslighter': gaslighter,
    'pushover': pushover,
    'north korea': north_korea,
    'Sam': Sam,
    'ryan': ryan,
    'Steve': Steve,
    'wenng': wenng,
    'fat_finger': fat_finger,
    'Con Air': Con_Air,
    'Saad': Saad,
    'punisher': punisher,
    'jon k': jon_k,
    'paffshank redemption': paffshank_redemption,
    'neo mar': neo_mar,
    'Julia Fractal': Julia,
    'ASR': defect,
    'MLN': M_Strategy,
    'eneguE': eneguE,
    'grudge holder': grudge_holder,
    'coolpant': sus_tit_for_tat,
    'JFK': my_adaptive_strategy,
    'Boaty McBoatface': boaty_mcboatface

}

# Run the prisoner's dilemma simulation
results, total_scores, matchup_scores = play_prisoners_dilemma(contestants, iterations=50)




In [ ]:
from IPython.display import display, HTML

# Display the score matrix with hyphens for self-intersections
matchup_scores_df = pd.DataFrame(matchup_scores).astype(int).T

# Function to apply color scale
def color_scale(val, row_name, col_name):
    if row_name == col_name:
        return 'background-color: black'
    # Define the range for the colors
    min_val, max_val = 0, matchup_scores_df.max().max()
    range_val = max_val - min_val
    if range_val == 0:
        return 'background-color: rgb(255,255,255)'  # Avoid division by zero
    # Calculate the color intensity based on the value
    intensity = (val - min_val) / range_val
    # Convert the intensity to a color
    r = int(255 * (1 - intensity))
    g = int(255 * intensity)
    b = 0
    return f'background-color: rgb({r},{g},{b})'

# Apply the color scale to the entire DataFrame
def apply_color_scale(df):
    styled_df = df.style.apply(lambda col: [
        color_scale(df.loc[idx, col.name], idx, col.name) for idx in df.index], axis=0)
    return styled_df

# Apply the color scale function to the DataFrame
styled_df = apply_color_scale(matchup_scores_df)

# Display the styled DataFrame
display(HTML("<h2>Matchup Scores</h2>"))
display(styled_df)


,gaslighter,pushover,north korea,Sam,ryan,Steve,wenng,fat_finger,Con Air,Saad,punisher,jon k,paffshank_redemption,neo mar,Julia Fractal,ASR,MLN,eneguE,grudge holder,coolpant,JFK,Boaty McBoatface
gaslighter,0,147,97,51,47,47,55,55,47,47,55,47,55,47,49,47,141,55,55,53,71,123
pushover,6,0,50,4,0,0,100,2,0,0,98,0,76,0,0,0,4,98,100,98,100,100
north korea,31,125,0,29,25,25,27,31,25,25,27,25,27,25,41,25,79,67,27,73,85,85
Sam,54,148,98,0,48,138,50,52,48,48,50,138,54,138,52,48,144,54,50,52,72,102
ryan,56,150,100,54,0,50,52,58,50,50,52,50,52,50,52,50,150,52,52,50,70,116
Steve,56,150,100,9,50,0,52,56,50,50,52,50,52,50,50,50,150,52,52,50,70,110
wenng,52,100,99,53,49,49,0,51,49,49,98,49,60,49,53,49,141,98,100,51,100,100
fat_finger,52,149,97,52,46,47,51,0,47,50,50,45,52,49,51,47,144,52,51,52,72,105
Con Air,56,150,100,54,50,50,52,56,0,50,52,50,52,50,52,50,150,52,52,50,70,116
Saad,56,150,100,54,50,50,52,50,50,0,52,50,52,50,50,50,150,52,52,50,70,106


In [ ]:
# Calculate the total score for each contestant
total_scores = matchup_scores_df.sum(axis=1)
total_scores.name = 'Total_Score'

# Create a new series with the contestant names and their total scores
total_scores_series = pd.Series(total_scores, name='Total_Score').sort_values(ascending=False)

# Display the total scores
display(HTML("<h2>Total Scores</h2>"))
display(total_scores_series.to_frame().style.apply(lambda x: [
    'background-color: gold' if x.name == total_scores_series.idxmax() else
    'background-color: silver' if x.name == total_scores_series.nlargest(2).idxmin() else
    'background-color: #cd7f32' if x.name == total_scores_series.nlargest(3).idxmin() else ''
], axis=1))

,Total_Score
Sam,1638
punisher,1511
wenng,1499
grudge holder,1497
eneguE,1477
ryan,1416
Con Air,1414
ASR,1404
Saad,1396
gaslighter,1391
